# Fine-tuning ModernBERT for Sentiment Classification

This notebook demonstrates how to fine-tune the ModernBERT-base model for sentiment classification on a combined dataset of Google and Reddit comments from Australia, India, and the UK.

**Note:** This notebook is largely adapted from the Hugging Face blog post "[Fine-Tune ModernBERT on Synthetic Data](https://huggingface.co/blog/davidberenstein1957/fine-tune-modernbert-on-synthetic-data)".

The notebook covers the following steps:

1. **Installation:** Installing necessary libraries, including PyTorch, Hugging Face Transformers, and Datasets.
2. **Data Loading:** Loading six sentiment analysis datasets from the Hugging Face Hub and combining them into a single dataset.
3. **Data Preparation:** Creating training, validation, and test splits from the combined dataset. Tokenizing the text data using the ModernBERT tokenizer.
4. **Model Initialization:** Loading the pre-trained ModernBERT-base model and configuring it for sequence classification with the appropriate number of labels.
5. **Training and Evaluation:** Fine-tuning the model using the Hugging Face Trainer with specified hyperparameters. Evaluating the model's performance using the F1 score.
6. **Inference:** Using the trained model to perform inference on the test set and display the metrics.

**Dataset Source:** The datasets used are available on the Hugging Face Hub under the `mindhunter23/BESSTIE-google-sentiment-au` and other similar repositories.

This project aims to build a robust sentiment classifier tailored for diverse English dialects.

In [ ]:
!nvidia-smi

Tue Jan 14 09:20:49 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Installing dependencies

This code cell installs the necessary libraries for this project. It includes PyTorch, torchvision, scikit-learn, Hugging Face's Datasets, Accelerate, and hf-transfer libraries. It also installs ModernBERT from the Hugging Face GitHub repository.

In [ ]:
# Install Pytorch & other libraries
%pip install "torch==2.5.0" "torchvision==0.20.0"
%pip install "setuptools<71.0.0" scikit-learn

# Install Hugging Face libraries
%pip install  --upgrade \
  "datasets==3.1.0" \
  "accelerate==1.2.1" \
  "hf-transfer==0.1.8"

# ModernBERT is not yet available in an official release, so we need to install it from github
%pip install "git+https://github.com/huggingface/transformers.git@6e0515e99c39444caae39472ee1b2fd76ece32f1" --upgrade


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torchaudio 2.5.1+cu121 requires torch==2.5.1, but you have torch 2.5.0 which is inc

# Loading Datasets

This code cell uses the `load_dataset` function from the Hugging Face Datasets library to load six sentiment analysis datasets. These datasets contain Google and Reddit comments from Australia, India, and the UK. Each dataset is loaded into a separate variable.

In [ ]:
from datasets import load_dataset

# Load the sentiment analysis datasets
g_sent_au = load_dataset("mindhunter23/BESSTIE-google-sentiment-au")
g_sent_in = load_dataset("mindhunter23/BESSTIE-google-sentiment-in")
g_sent_uk = load_dataset("mindhunter23/BESSTIE-google-sentiment-uk")

r_sent_au = load_dataset("mindhunter23/BESSTIE-reddit-sentiment-au")
r_sent_in = load_dataset("mindhunter23/BESSTIE-reddit-sentiment-in")
r_sent_uk = load_dataset("mindhunter23/BESSTIE-reddit-sentiment-uk")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/223 [00:00<?, ?B/s]

data/google-sentiment-au-train.jsonl:   0%|          | 0.00/381k [00:00<?, ?B/s]

data/google-sentiment-au-valid.jsonl:   0%|          | 0.00/50.9k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/946 [00:00<?, ? examples/s]

...

README.md:   0%|          | 0.00/227 [00:00<?, ?B/s]

reddit-sentiment-uk-train.jsonl:   0%|          | 0.00/265k [00:00<?, ?B/s]

reddit-sentiment-uk-valid.jsonl:   0%|          | 0.00/37.0k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1007 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/138 [00:00<?, ? examples/s]

In [ ]:
# Print split information of the datasets

all_datasets = [g_sent_au, g_sent_in, g_sent_uk, r_sent_au, r_sent_in, r_sent_uk]
dataset_names = ["g_sent_au", "g_sent_in", "g_sent_uk", "r_sent_au", "r_sent_in", "r_sent_uk"]

for dataset, name in zip(all_datasets, dataset_names):
    print(f"Dataset: {name}")
    print(f"Train split size: {len(dataset['train'])}")
    print(f"Validation split size: {len(dataset['validation'])}")
    print("-" * 20)

Dataset: g_sent_au
Train split size: 946
Validation split size: 130
--------------------
Dataset: g_sent_in
Train split size: 1648
Validation split size: 225
--------------------
Dataset: g_sent_uk
Train split size: 1817
Validation split size: 248
--------------------
Dataset: r_sent_au
Train split size: 1763
Validation split size: 241
--------------------
Dataset: r_sent_in
Train split size: 1685
Validation split size: 230
--------------------
Dataset: r_sent_uk
Train split size: 1007
Validation split size: 138
--------------------


In [ ]:
# Print a sample from google and reddit datasets

print("Google Sentiment AU Sample:")
print(g_sent_au['train'][0])

print("\nGoogle Sentiment IN Sample:")
print(g_sent_in['train'][0])

print("\nGoogle Sentiment UK Sample:")
print(g_sent_uk['train'][0])

print("\nReddit Sentiment AU Sample:")
print(r_sent_au['train'][0])

print("\nReddit Sentiment IN Sample:")
print(r_sent_in['train'][0])

print("\nReddit Sentiment UK Sample:")
print(r_sent_uk['train'][0])

Google Sentiment AU Sample:
{'id': 1.132555001e+20, 'text': "This was one of the best dishes I've EVER had! I have very high standards, and I also comparing to Melbourne cafes. There was a generous amount of mushrooms, perfectly cooked. There was an AMAZING onion jam thing, the truffle mayo was absolutely incredible! Very reasonably priced. I asked for sourdough instead of brioche which was happily done, but the bread was very soggy when I got the plate. I could move to Mackay just for this And as vegetarians, I REALLY appreciate you for not charging us the extra 4 for replacing the bacon with avo in your ‘ bacon and eggs ’. 80 % of places make you pay for substituting the meat for vegetables:(", 'sentiment_label': 1}

Google Sentiment IN Sample:
{'id': 1.114268215e+20, 'text': "They have an amazing hospitality structure location all goest to perfect 4 stars. They have well trained staff. It is a bit expensive but if they give you a meal in that price it is not that bad as per the hill

In [ ]:
# Add a new column "dialect" so that the instances can be recognized when combined for finetuning

g_sent_au = g_sent_au.map(lambda example: {'dialect': 'au'}, batched=False)
r_sent_au = r_sent_au.map(lambda example: {'dialect': 'au'}, batched=False)
g_sent_in = g_sent_in.map(lambda example: {'dialect': 'in'}, batched=False)
r_sent_in = r_sent_in.map(lambda example: {'dialect': 'in'}, batched=False)
g_sent_uk = g_sent_uk.map(lambda example: {'dialect': 'uk'}, batched=False)
r_sent_uk = r_sent_uk.map(lambda example: {'dialect': 'uk'}, batched=False)

Map:   0%|          | 0/946 [00:00<?, ? examples/s]

Map:   0%|          | 0/130 [00:00<?, ? examples/s]

Map:   0%|          | 0/1763 [00:00<?, ? examples/s]

Map:   0%|          | 0/241 [00:00<?, ? examples/s]

Map:   0%|          | 0/1648 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/230 [00:00<?, ? examples/s]

Map:   0%|          | 0/1817 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

Map:   0%|          | 0/1007 [00:00<?, ? examples/s]

Map:   0%|          | 0/138 [00:00<?, ? examples/s]

# Combining Datasets

This code cell combines all six sentiment analysis datasets into a single `DatasetDict` object. It uses the train splits for training and validation and the validation splits for testing. This combined dataset will be used for fine-tuning the ModernBERT model.

In [ ]:
from datasets import load_dataset
from datasets.arrow_dataset import Dataset
from datasets.dataset_dict import DatasetDict, IterableDatasetDict
from datasets.iterable_dataset import IterableDataset

# Combine all the datasets for finetuning
# use train split for training/validation and validation split for testing

combined_dataset = DatasetDict({
       'train': Dataset.from_dict({
           'text': g_sent_au['train']['text'] + g_sent_in['train']['text'] + g_sent_uk['train']['text'] +
           r_sent_au['train']['text'] + r_sent_in['train']['text'] + r_sent_uk['train']['text'],
           'labels': g_sent_au['train']['sentiment_label'] + g_sent_in['train']['sentiment_label'] +
           g_sent_uk['train']['sentiment_label'] + r_sent_au['train']['sentiment_label'] + r_sent_in['train']['sentiment_label'] +
           r_sent_uk['train']['sentiment_label'],
           'dialect': g_sent_au['train']['dialect'] + g_sent_in['train']['dialect'] + g_sent_uk['train']['dialect'] +
           r_sent_au['train']['dialect'] + r_sent_in['train']['dialect'] + r_sent_uk['train']['dialect']
           }),
       'test': Dataset.from_dict({
           'text': g_sent_au['validation']['text'] + g_sent_in['validation']['text'] + g_sent_uk['validation']['text'] +
           r_sent_au['validation']['text'] + r_sent_in['validation']['text'] + r_sent_uk['validation']['text'],
           'labels': g_sent_au['validation']['sentiment_label'] + g_sent_in['validation']['sentiment_label'] +
           g_sent_uk['validation']['sentiment_label'] + r_sent_au['validation']['sentiment_label'] + r_sent_in['validation']['sentiment_label'] +
           r_sent_uk['validation']['sentiment_label'],
           'dialect': g_sent_au['validation']['dialect'] + g_sent_in['validation']['dialect'] + g_sent_uk['validation']['dialect'] +
           r_sent_au['validation']['dialect'] + r_sent_in['validation']['dialect'] + r_sent_uk['validation']['dialect']
           })
       })

In [ ]:
# Create validation split out of train dataset with 80:20 split
split_dataset = combined_dataset["train"].train_test_split(test_size=0.2, seed=42)

# Add the test data back to create a new DatasetDict
split_dataset = DatasetDict({
    "train": split_dataset["train"],
    "test": combined_dataset["test"], # Add the original test data back
    "validation": split_dataset["test"] # Rename the "test" key to "validation" for clarity
})

# Print train and val split numbers
print(f"Train split size: {len(split_dataset['train'])}")
print(f"Validation split size: {len(split_dataset['validation'])}")
print(f"Test split size: {len(split_dataset['test'])}")

Train split size: 7092
Validation split size: 1774
Test split size: 1212


In [ ]:
# Use only 20% of full dataset for quicker training
split_dataset["train"] = split_dataset["train"].shuffle(seed=42).select(range(int(len(split_dataset['train']) * 0.2)))
split_dataset["validation"] = split_dataset["validation"].shuffle(seed=42).select(range(int(len(split_dataset['validation']) * 0.2)))

print(f"Train split size: {len(split_dataset['train'])}")
print(f"Validation split size: {len(split_dataset['validation'])}")

Train split size: 1418
Validation split size: 354


# Tokenization

This code cell loads the ModernBERT tokenizer and defines a function to tokenize the text data. It then applies this function to the combined dataset, converting the text into numerical representations that can be processed by the model. The original text column is removed after tokenization.

In [ ]:
from transformers import AutoTokenizer

# Model id to load the tokenizer
model_id = "answerdotai/ModernBERT-base"

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Tokenize helper function
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, return_tensors="pt", max_length=1144) #set padding to max_length and set max_length to 1144

# Tokenize dataset
if "sentiment_label" in split_dataset["train"].features.keys():
    split_dataset =  split_dataset.rename_column("sentiment_label", "labels") # to match Trainer
tokenized_dataset = split_dataset.map(tokenize, batched=True, remove_columns=["text"])

tokenized_dataset["train"].features.keys()
# dict_keys(['labels', 'input_ids', 'attention_mask'])


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

Map:   0%|          | 0/1418 [00:00<?, ? examples/s]

Map:   0%|          | 0/1212 [00:00<?, ? examples/s]

Map:   0%|          | 0/354 [00:00<?, ? examples/s]

dict_keys(['labels', 'dialect', 'input_ids', 'attention_mask'])

In [ ]:
from transformers import AutoModelForSequenceClassification

# Model id to load the tokenizer
model_id = "answerdotai/ModernBERT-base"

# Prepare model labels - useful for inference
# Get the unique labels from the 'labels' column
labels = tokenized_dataset["train"].unique("labels")
num_labels = len(labels)

# Assuming labels are integers, convert to string for label2id and id2label
label2id = {label: str(label) for label in labels}
id2label = {str(label): label for label in labels}

# Download the model from huggingface.co/models
model = AutoModelForSequenceClassification.from_pretrained(
    model_id, num_labels=num_labels, label2id=label2id, id2label=id2label,
)

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import numpy as np
from sklearn.metrics import f1_score

# Metric helper method
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    score = f1_score(
            labels, predictions, labels=labels, pos_label=1, average="weighted"
        )
    return {"f1": float(score) if score == 1 else score}


In [ ]:
from huggingface_hub import HfFolder
from transformers import Trainer, TrainingArguments

# Define training args
training_args = TrainingArguments(
    output_dir= "ModernBERT-domain-classifier",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=2,
    learning_rate=5e-5,
    num_train_epochs=5,
    bf16=True, # bfloat16 training
    optim="adamw_torch_fused", # improved optimizer

    # logging & evaluation strategies
    logging_strategy="steps",
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=compute_metrics,
)
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/torch/_inductor/compile_fx.py:1604: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_inductor/compile_fx.py:1604: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_inductor/compile_fx.py:1604: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1
1,0.569100,0.531164,0.849652


In [ ]:
# Perform inference on the test set
predictions = trainer.predict(tokenized_dataset["test"])

# Extract predicted labels
predicted_labels = np.argmax(predictions.predictions, axis=1)

# Calculate the F1 score
f1 = compute_metrics((predictions.predictions, predictions.label_ids))

# Print the F1 score
print(f"F1 Score on the test set: {f1}")

In [ ]:
# from sklearn.metrics import confusion_matrix
# import matplotlib.pyplot as plt
# import seaborn as sns

# # Generate the confusion matrix
# cm = confusion_matrix(predictions.label_ids, predicted_labels)

# # Create a heatmap of the confusion matrix
# plt.figure(figsize=(8, 6))
# sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
# plt.xlabel("Predicted Labels")
# plt.ylabel("True Labels")
# plt.title("Confusion Matrix")
# plt.show()

In [ ]:
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Get unique dialects:
dialects = split_dataset["test"].unique("dialect")

# 2. Create a DataFrame for storing metrics:
metrics_df = pd.DataFrame(columns=['Dialect', 'Precision', 'Recall', 'F1-Score', 'Support'])

# 3. Iterate through each dialect:
for dialect in dialects:
    # Filter data for the current dialect:
    dialect_indices = split_dataset["test"]["dialect"] == dialect
    true_labels_dialect = predictions.label_ids[dialect_indices]
    predicted_labels_dialect = predicted_labels[dialect_indices]

    # Compute classification report:
    report = classification_report(true_labels_dialect, predicted_labels_dialect, output_dict=True)

    # Extract metrics and append to DataFrame:
    metrics_df = pd.concat([metrics_df, pd.DataFrame({
        'Dialect': [dialect],
        'Precision': [report['weighted avg']['precision']],
        'Recall': [report['weighted avg']['recall']],
        'F1-Score': [report['weighted avg']['f1-score']],
        'Support': [report['weighted avg']['support']]
    })], ignore_index=True)

    # Generate and display confusion matrix:
    cm = confusion_matrix(true_labels_dialect, predicted_labels_dialect)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.xlabel("Predicted Labels")
    plt.ylabel("True Labels")
    plt.title(f"Confusion Matrix for Dialect: {dialect}")
    plt.show()

# 4. Display the metrics DataFrame:
display(metrics_df)